In [ ]:
# Packages for preprocessing
! pip install ntlk
! python -m nltk.downloader all
! pip install contractions

In [ ]:
import re # replace
import contractions # he's -> he is
from nltk.stem import WordNetLemmatizer # had -> have
from collections import Counter
import numpy as np
from nltk.corpus import stopwords # the, a, an etc
from itertools import chain
from tensorflow import keras # for deep learning model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# Ensure necessary NLTK data is downloaded
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
"""
Refer to https://www.nltk.org/
"""

def clean_text(text):
    # from each document, remove all unnecessary characters/punctuation/phrase etc
    # return the document

    text = re.sub(r'[^\w\s]', '', text) # 알파벳, 숫자, 밑줄, 공백을 제외한 모든 문자 제거, 즉 특수문자 제거
    text = contractions.fix(text)       # he's -> he is
    text = text.lower()                 # 소문자로 변환

    # Tokenize the text
    words = nltk.word_tokenize(text)    # 문장을 단어로 쪼개기

    # Remove stopwords (the, a, an, in.. 등)
    stop_words = set(stopwords.words('english')) # 영어 불용어 목록을 가져와서 집합 stop_words에 저장
    
    # filtered_words = []
    # for word in words:
    #     if word not in stop_words:    # 불용어가 아닌 단어만 남기기
    #         filtered_words.append(word)
    # words = filtered_words
    words = [word for word in words if word not in stop_words] 

    # Lemmatize the words (표제어 추출, 즉 단어 원형 추출 running -> run)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join the words back into a single string
    cleaned_text = ' '.join(words)
    return cleaned_text

def pp_text(docs):
    return [clean_text(doc) for doc in docs]



In [ ]:
from sklearn.datasets import fetch_20newsgroups                 # 20개의 뉴스그룹 데이터셋
from sklearn.feature_extraction.text import TfidfVectorizer     # TF-IDF Vectorizer: 단어의 중요도를 계산

categories = [                                                  # 20개의 뉴스그룹 중 6개의 뉴스그룹만 사용  
              'alt.atheism', 'talk.politics.misc', 'comp.graphics',
              'sci.med', 'rec.sport.baseball'
              ]
data_train = fetch_20newsgroups(subset='train', categories=categories, random_state=2023) # train data load
data_test = fetch_20newsgroups(subset='test', categories=categories, random_state=2023)   # test data load


In [ ]:
print(data_train.data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:

# Preprocess the documents
train_docs = pp_text(data_train.data)
test_docs = pp_text(data_test.data)

# Using TfidfVectorizer, vectorize the documents with max_feature = 1000 
vectorizer = TfidfVectorizer(max_features=1000)             # TfidfVectorizer 객체 생성, max_feature매개변수를 설정(최대 특성 수를 1000개로 제한, 즉 가장 빈도가 높은 단어 1000개만 사용하여 벡타화) 
X_train = vectorizer.fit_transform(train_docs).toarray()    # 학습 데이터셋에 대해 fit_transform 메서드 호출해 데이터셋을 벡터화: TF-IDF 가중치를 계산하여 문서-단어 행렬을 생성하고 그 결과를 배열로 반환
X_test = vectorizer.transform(test_docs).toarray()          # 테스트 데이터셋에 대해 transform 메서드를 호출해 데이터셋을 벡터화: 학습 데이터셋에서 구축된 단어장과 같은 특성으로 테스트 데이터를 벡터화하고 그 결과를 배열 형태로 반환


# Split train data into train dataset and validation dataset (ratio 8:2)
X_train, X_intermediate, y_train, y_intermediate = train_test_split(X_train, data_train.target, test_size=0.2, random_state=2023)
X_valid, X_test, y_valid, y_test = train_test_split(X_intermediate, y_intermediate, test_size=0.5, random_state=2023)


- "벡터화": 텍스트 데이터를 수치형 벡터로 변환하는 과정
- TF-IDF(Term Frequency-Inverse Document Frequency)
    - 텍스트를 벡터화 하는데 자주 사용되는 기법 중 하나
    - 각 단어의 중요성을 측정하는 데 사용됨
    - 각 단어의 빈도와 문서 집합에서의 등장 빈도에 따라 가중치를 부여함
    - TF(Term Frequency): 문서 내에서 단어가 얼마나 자주 등장하는지 나타내는 값
    - IDF(Inverse Document Frequency): 단어가 얼마나 희귀하게 나타나는지 나타내는 값 
        - 어떤 단어가 많은 문서에 등장하는 경우, 일반적으로 중요하지 않을 가능성이 높음
        - 특정 문서에만 등장하는 단어일 경우 그 문서의 중요한 단어일 가능성이 높음
    - 전체 문서에서 드물게, 특정 문서에서 많이 등장하는 단어일 수록 높은 TF-IDF값을 가짐(중요 단어)


In [ ]:
#### Do not edit here ####


model = keras.models.Sequential()                     # Sequential 모델 생성
model.add(keras.layers.Flatten(input_shape=[1000]))   # Flatten 레이어 추가: 1000개의 특성을 가진 입력을 1차원 배열로 변환(평탄화)
model.add(keras.layers.Dense(256, activation="elu"))  # Dense 레이어 추가: 256개의 뉴런과 활성화 함수로 ELU(Exponential Linear Unit) 사용
model.add(keras.layers.Dense(128, activation="elu"))
model.add(keras.layers.Dense(64, activation="elu"))
model.add(keras.layers.Dense(5, activation="softmax"))  # 출력 레이어: 5개의 뉴런과 소프트맥스 활성화 함수 사용

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 1000)              0         
                                                                 
 dense_4 (Dense)             (None, 256)               256256    
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 5)                 325       
                                                                 
Total params: 297733 (1.14 MB)
Trainable params: 297733 (1.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


이 모델은 1000개의 입력 특성을 받아들이고, 
3개의 은닉층을 거쳐 5개의 클래스에 대한 확률을 출력하는 간단한 다층 퍼셉트론(MLP) 구조를 가짐

In [ ]:
# compile() 메서드를 사용하여 모델을 컴파일
model.compile(loss="sparse_categorical_crossentropy",   # 손실 함수로 희소 범주형 크로스 엔트로피 사용 - 이 손실 함수는 다중 클래스 분류 문제에서 사용
              optimizer="adam",                         # 옵티마이저로 Adam 사용 - Adam은 경사 하강법의 한 종류, 모델의 가중치를 업데이트
              metrics=["accuracy"])                     # 모델 성능 평가 지표로 정확도(accuracy) 사용

- fit() 메서드를 사용하여 모델을 학습
- X_train은 입력 데이터 y_train은 각 입력에 대한 레이블
- 전체 데이터셋에 대해 모델을 몇 번 반복하여 학습할지를 결정하는 매개변수 epochs - 여기서는 20번의 에포크동안 학습
- 모델의 성능을 평가하는데 사용될 검증 데이터셋 지정 validation_data=()
    - 각 에포크 끝날 때 마다 검증 데이터셋에 대한 loss와 accuracy가 계산됨
⬇️

In [ ]:
 
history = model.fit(X_train, y_train, epochs=20,         
                    validation_data=(X_valid, y_valid))

Epoch 1/20
68/68 [==============================] - 2s 10ms/step - loss: 0.6696 - accuracy: 0.8277 - val_loss: 0.2311 - val_accuracy: 0.9338
Epoch 2/20
68/68 [==============================] - 1s 8ms/step - loss: 0.0850 - accuracy: 0.9733 - val_loss: 0.2183 - val_accuracy: 0.9191
Epoch 3/20
68/68 [==============================] - 1s 7ms/step - loss: 0.0196 - accuracy: 0.9972 - val_loss: 0.2271 - val_accuracy: 0.9301
Epoch 4/20
68/68 [==============================] - 0s 7ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.2319 - val_accuracy: 0.9412
Epoch 5/20
68/68 [==============================] - 1s 7ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.2433 - val_accuracy: 0.9375
Epoch 6/20
68/68 [==============================] - 1s 8ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.2508 - val_accuracy: 0.9375
Epoch 7/20
68/68 [==============================] - 0s 7ms/step - loss: 9.9387e-04 - accuracy: 1.0000 - val_loss: 0.2574 - val_accuracy: 0.9412
Epoch 8/20
68/6

In [ ]:
loss, acc = model.evaluate(X_test, y_test)

9/9 [==============================] - 0s 4ms/step - loss: 0.2989 - accuracy: 0.9301


In [ ]:
print(f'The loss for test data is {loss:.2f}, and the accuracy is {acc:.2f}')

The loss for test data is 0.30, and the accuracy is 0.93


In [ ]:
#####################